In [16]:
# !pip install -U sacrebleu evaluate datasets transformers

In [17]:

# import kagglehub
# alvations_tatoeba_path = kagglehub.dataset_download('alvations/tatoeba')

# print('Data source import complete.')


In [ ]:
import zhconv
import torch
import sentencepiece as spm
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, MarianConfig



In [19]:
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

# def preprocess_function(batch):
#     batch['SRC'] = zhconv.convert(batch['SRC'], 'zh-cn')
#     inputs = tokenizer(batch['SRC'], max_length=128, truncation=True, padding="max_length")
#     outputs = tokenizer(batch['TRG'], max_length=128, truncation=True, padding="max_length")

#     return {"input_ids": inputs["input_ids"],
#             "attention_mask": inputs["attention_mask"],
#             "decoder_input_ids": outputs["input_ids"],
#             "decoder_attention_mask": outputs["attention_mask"],
#             "labels": outputs.input_ids.copy()}



In [20]:
# # 加载 SentencePiece 模型
# sp = spm.SentencePieceProcessor()
# sp.load("spm_zh_en.model")

# # 定义一个函数，用于批量转换
# def batch_convert_to_simplified(text_list):
#     simplified_list = [zhconv.convert(text, 'zh-cn') for text in text_list]
#     return simplified_list

# def preprocess_function(batch):
#     # 转换为简体中文
#     src = batch_convert_to_simplified(batch['SRC'])
#     trg = batch['TRG']

#     # 添加目标语言标记
#     src = [">>eng<< " + source for source in src]

#     # 使用 sentencepiece 进行分词和编码
#     input_ids = sp.encode(src, out_type=int)
#     decoder_input_ids = sp.encode(trg, out_type=int)

#     # 生成 attention mask（非PAD为1）
#     input_attention_mask = [1] * len(input_ids)
#     decoder_attention_mask = [1] * len(decoder_input_ids)

#     # 构造 label（通常和 decoder_input_ids 一致，但训练时需要去掉 decoder_input_ids 的最后一个token）
#     labels = decoder_input_ids.copy()

#     # 补齐到最大长度（例如128），使用0作为pad
#     max_len = 128
#     pad_id = 0

#     def pad(seq):
#         return seq + [pad_id] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len]

#     return {
#         "input_ids": torch.tensor(pad(input_ids), dtype=torch.long),
#         "attention_mask": torch.tensor(pad(input_attention_mask), dtype=torch.long),
#         "decoder_input_ids": torch.tensor(pad(decoder_input_ids), dtype=torch.long),
#         "decoder_attention_mask": torch.tensor(pad(decoder_attention_mask), dtype=torch.long),
#         "labels": torch.tensor(pad(labels), dtype=torch.long),
#     }


In [21]:
import torch
import zhconv
import sentencepiece as spm

# 加载 SentencePiece 模型
sp = spm.SentencePieceProcessor()
sp.load("spm_zh_en.model")

# 定义一个函数，用于批量转换为简体中文
def batch_convert_to_simplified(text_list):
    return [zhconv.convert(text, 'zh-cn') for text in text_list]

# 补齐函数：对每个序列进行 padding 或截断
def pad_sequence(seq, max_len=128, pad_id=0):
    return seq + [pad_id] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len]

def preprocess_function(batch):
    # 批量转换简体中文
    src_list = batch_convert_to_simplified(batch['SRC'])
    trg_list = batch['TRG']

    # 添加语言标识
    src_list = [">>eng<< " + text for text in src_list]

    # 编码（sentencepiece）
    input_ids_batch = [sp.encode(text, out_type=int) for text in src_list]
    decoder_input_ids_batch = [sp.encode(text, out_type=int) for text in trg_list]

    # 补齐并生成 attention mask
    max_len = 128
    pad_id = 0

    def process_batch(sequences):
        padded = [pad_sequence(seq, max_len, pad_id) for seq in sequences]
        mask = [[1 if token != pad_id else 0 for token in seq] for seq in padded]
        return padded, mask

    input_ids, input_attention_mask = process_batch(input_ids_batch)
    decoder_input_ids, decoder_attention_mask = process_batch(decoder_input_ids_batch)
    labels = decoder_input_ids  # labels与decoder_input_ids相同

    # 转为 tensor（batch_size, seq_len）
    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(input_attention_mask, dtype=torch.long),
        "decoder_input_ids": torch.tensor(decoder_input_ids, dtype=torch.long),
        "decoder_attention_mask": torch.tensor(decoder_attention_mask, dtype=torch.long),
        "labels": torch.tensor(labels, dtype=torch.long),
    }


In [ ]:
tatoeba_dataset = load_dataset("csv", data_files="tatoeba-sentpairs.tsv",
                  streaming=False, delimiter="\t", encoding="utf-8", split="train")

print(tatoeba_dataset)
tatoeba_dataset = tatoeba_dataset.filter(lambda x: x['SRC LANG'] == "cmn" and x['TRG LANG'] == 'eng')
tatoeba_dataset = tatoeba_dataset.map(preprocess_function, batched=True)

In [23]:
split_dataset = tatoeba_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)

# 从训练集选前100条
train_dataset = split_dataset['train']  # .select(range(50))

# 从测试集选前10条
test_dataset = split_dataset['test']  # .select(range(10))


In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [25]:
# mt_metrics = evaluate.combine(
#     ["bleu", "chrf"]
# )

# def compute_metrics(pred):
#     labels_ids = pred.label_ids
#     pred_ids = pred.predictions

#     predictions = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

#     labels_ids[labels_ids == -100] = tokenizer.pad_token_id
#     references = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

#     outputs = mt_metrics.compute(predictions=predictions,
#                              references=references)

#     return outputs

In [ ]:
import evaluate

mt_metrics = evaluate.combine(
    ["bleu", "chrf"]
)


def compute_metrics(pred):
    pad_id = sp.pad_id()  # 获取 SentencePiece 中的 pad_id，通常是 0

    # 获取预测和标签
    pred_ids = pred.predictions
    labels_ids = pred.label_ids

    # 将 labels 中的 -100 替换为 pad_id（避免 decode 报错）
    cleaned_labels = [[token if token != -100 else pad_id for token in label] for label in labels_ids]

    # 将 numpy/tensor 转为 list，并用 SentencePiece 解码
    predictions = sp.decode_batch(pred_ids.tolist())
    references = sp.decode_batch(cleaned_labels)

    # 可选：去除特殊标记，比如 >>eng<<
    def clean_text(text):
        return text.replace(">>eng<<", "").strip()

    predictions = [clean_text(text) for text in predictions]
    references = [clean_text(text) for text in references]

    # 计算 BLEU、chrF 等指标
    outputs = mt_metrics.compute(predictions=predictions, references=references)
    return outputs


In [ ]:
# model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

# # set special tokens, not sure if it's needed but adding them for sanity...
# model.config.eos_token_id = tokenizer.eos_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

In [12]:
model = MarianMTModel.from_pretrained("trans-opus-mt-zh-en")

# set special tokens, not sure if it's needed but adding them for sanity...
model.config.eos_token_id = sp.eos_id()
model.config.pad_token_id = sp.pad_id()


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    save_steps=5,
    eval_steps=1,
    # max_steps=10,
    eval_strategy="steps",  # evaluation_strategy
    predict_with_generate=True,
    report_to=None,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.with_format("torch"),
    eval_dataset=test_dataset.with_format("torch"),
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
# # 手动保存 tokenizer，如果用transformer的tokenizer
# tokenizer.save_pretrained('./')

## 训练分词模型

In [ ]:
!pip install sentencepiece
!pip install -U datasets

In [ ]:
!spm_train \
  --input=output.txt \
  --model_prefix=spm_zh_en \
  --vocab_size=8000 \
  --character_coverage=1.0 \
  --model_type=bpe \
  --pad_id=0 \
  --unk_id=1 \
  --bos_id=2 \
  --eos_id=3 \
  --user_defined_symbols=["<s>", "</s>", "<cmd>"]


In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input='total_output.txt',
    model_prefix='spm_zh_en',
    vocab_size=32000,
    character_coverage=1.0,
    model_type='bpe',
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    user_defined_symbols='["<s>", "</s>", "<cmd>"]'
)


In [ ]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load("spm_zh_en.model")

text = "今天天气不错，Let's go shopping!"
pieces = sp.encode(text, out_type=str)
ids = sp.encode(text, out_type=int)

print("Pieces:", pieces)
print("IDs:", ids)

# 解码
decoded = sp.decode(ids)
print("Decoded:", decoded)


In [ ]:
# pip install -q datasets
from datasets import load_dataset
ds = load_dataset("Muennighoff/xP3x", "zho_Hans", streaming=False) # Use streaming to not download all at once


In [ ]:
import random

# 随机打乱索引后选50个
total_len = len(ds['train'])
random_indices = random.sample(range(total_len), 1000000)  # 随机选一百万条数据

subset = ds['train'].select(random_indices)


In [ ]:
zh_txt = []
count = 0
for text in subset:
  count += 1
  zh_txt.append(text['targets'])

  if count % 10000 == 0:
    print(count/10000)

In [ ]:
with open('zh_txt.txt', 'w', encoding='utf-8') as f:
  for line in zh_txt:
    f.write(line.strip() + '\n')

In [ ]:
from datasets import load_dataset

# 流式加载 eng_Latn 子集
ds_en = load_dataset("Muennighoff/xP3x", "eng_Latn", streaming=True, split="train")

# 处理函数（根据你的任务修改）
def process(example):
    # 示例：拼接 inputs 和 targets
    return example['targets']

en_txt = []
# 保存到文件或直接处理
with open("en_txt.txt", "w", encoding="utf-8") as f:
    for i, example in enumerate(ds_en):
        line = process(example)
        f.write(line + "\n")
        # 可选：打印进度
        if i % 10000 == 0:
            print(f"Processed {i/10000} examples")

        if i == 1000000:
          break

In [ ]:
import re

# 定义一个函数，用于判断文本是否包含中文或英文
def is_valid_text(text):
    # 使用正则表达式匹配中文或英文字符
    if re.search(r'[\u4e00-\u9fff]', text) or re.search(r'[a-zA-Z]', text):
        return True
    return False

# 输入文件路径和输出文件路径
input_file_path = '/content/zh_txt.txt'  # 替换为你的输入文件路径
output_file_path = '/content/zh_txt_output.txt'  # 输出文件路径

# 打开输入文件和输出文件
with open(input_file_path, 'r', encoding='utf-8') as infile, \
     open(output_file_path, 'w', encoding='utf-8') as outfile:
    # 逐行读取文件
    for line in infile:
        # 去除行首和行尾的空白字符
        line = line.strip()
        # 如果行包含中文或英文，则写入输出文件
        if is_valid_text(line):
            outfile.write(line + '\n')

print(f"处理完成，结果已保存到 {output_file_path}")

In [ ]:
# 输入文件路径和输出文件路径
input_file_path = '/content/en_txt.txt'  # 替换为你的输入文件路径
output_file_path = '/content/en_txt_output.txt'  # 输出文件路径

# 使用集合来存储已经见过的行
seen_lines = set()

# 打开输入文件和输出文件
with open(input_file_path, 'r', encoding='utf-8') as infile, \
     open(output_file_path, 'w', encoding='utf-8') as outfile:
    # 逐行读取文件
    for line in infile:
        # 去除行首和行尾的空白字符
        line = line.strip()
        # 如果这行还没有出现过，则写入输出文件，并将其加入集合
        if line not in seen_lines:
            outfile.write(line + '\n')
            seen_lines.add(line)

print(f"去重完成，结果已保存到 {output_file_path}")

In [ ]:
import random

# 输入文件路径
input_file_path1 = '/content/en_txt_output.txt'  # 第一个文件路径
input_file_path2 = '/content/zh_txt_output.txt'  # 第二个文件路径
output_file_path = 'total_output.txt'  # 输出文件路径

# 读取第一个文件的内容到列表1
with open(input_file_path1, 'r', encoding='utf-8') as file1:
    list1 = file1.read().splitlines()

# 读取第二个文件的内容到列表2
with open(input_file_path2, 'r', encoding='utf-8') as file2:
    list2 = file2.read().splitlines()

# 将两个列表合并
combined_list = list1 + list2[:600000]

# 随机打乱合并后的列表
random.shuffle(combined_list)

# 将打乱后的列表保存到新的文件中
with open(output_file_path, 'w', encoding='utf-8') as outfile:
    for line in combined_list:
        outfile.write(line + '\n')

print(f"两个文件的内容已随机混合并保存到 {output_file_path}")

In [ ]:
import re

# 输入文件路径和输出文件路径
input_file_path = '/content/total_output.txt'  # 替换为你的输入文件路径
output_file_path = 'output.txt'  # 输出文件路径

# 定义正则表达式，匹配仅包含中文、英文、常见标点符号和空白字符的行
# \u4e00-\u9fff 是中文字符范围，a-zA-Z 是英文字母范围
# \s 是空白字符，\u3000-\u303f 是中文标点符号范围，\u2000-\u206f 是通用标点符号范围
pattern = re.compile(r'^[\u4e00-\u9fffA-Za-z\s\u3000-\u303f\u2000-\u206f]*$')

# 打开输入文件和输出文件
with open(input_file_path, 'r', encoding='utf-8') as infile, \
     open(output_file_path, 'w', encoding='utf-8') as outfile:
    # 逐行读取文件
    for line in infile:
        # 去除行首和行尾的空白字符
        line = line.strip()
        # 如果行只包含中文、英文、标点符号和空白字符，则写入输出文件
        if pattern.match(line):
            outfile.write(line + '\n')

print(f"处理完成，结果已保存到 {output_file_path}")